<a href="https://colab.research.google.com/github/Yurii-Bortsov/svetofor/blob/main/cyberimmunity_traffic_lights_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Кибериммунный подход к разработке. Учебный пример "Светофор"

## Об авторе

Этот блокнот разработан для вас Сергеем Соболевым, sergey.p.sobolev@kaspersky.com

Больше информации о кибериммунном подходе можно найти на странице https://github.com/sergey-sobolev/cyberimmune-systems/wiki/%D0%9A%D0%B8%D0%B1%D0%B5%D1%80%D0%B8%D0%BC%D0%BC%D1%83%D0%BD%D0%B8%D1%82%D0%B5%D1%82

Подписывайтесь на телеграм-канал @learning_cyberimmunity (https://t.me/learning_cyberimmunity)

Обучающие видео на тему кибериммунного подхода вы можете найти на youtube канале https://www.youtube.com/@learning_cyberimmunity/

## О примере

Светофор - это, на первый взгляд, очень простая система, но она оказывает критическое влияние на безопасность дорожного движения.
Применим идеи конструктивной безопасности к архитектуре и реализации прототипа светофора. За отправную точку возьмём [код базового примера](https://colab.research.google.com/github/sergey-sobolev/cyberimmune-systems-basic-demo-notebook01/blob/main/cyberimmunity-basics.ipynb) и немного его переработаем.

Для переработки будем использовать описание архитектуры решения, которое обсуждалось на занятии.

## Простая реализация выбранной политики архитектуры

Будем использовать политику архитектуры, показанную на рис. 1.

![Рис. 1. Политика архитектуры](https://github.com/cyberimmunity-edu/cyberimmune-systems-example-traffic-light-jupyter-notebook/blob/master/images/tl-archpol-0.02.png?raw=1)

Рис. 1. Политика архитектуры светофора

1. Создадим функциональные компоненты (сущности 1-4) и монитор безопасности, который будет контролировать их взаимодействие, в том числе реализовывать контроль конфигураций светофора (сущность №5 на архитектурной диаграмме)
2. Определим политики безопасности
3. Сымитируем запрос на изменение режима для проверки работы всех элементов

- В качестве интерфейса взаимодействия используем очереди сообщений, у каждой сущности есть своя «персональная» очередь, ассоциированная с ней
- Компоненты 1-4 отправляют сообщения только в очередь monitor сущности SecurityMonitor
- SecurityMonitor проверяет сообщения на соответствие политикам безопасности, в случае положительного решения перенаправляет сообщение в очередь соответствующей сущности

В коде назовём сущности следующим образом
1. Связь - CitySystemConnector
2. Система управления светофора - ControlSystem
3. Управление светодиодами - LightsGPIO
4. Система диагностики - SelfDiagnosticsSystem

Логику контроля режимов светофора (компонент №5 на рис. 1) реализуем в виде политик безопасности в мониторе безопасности

![Рис. 2. Политика архитектуры с именами классов](https://github.com/cyberimmunity-edu/cyberimmune-systems-example-traffic-light-jupyter-notebook/blob/master/images/tl-archpol-code.png?raw=1)

Рис. 2. Политика архитектуры с именами классов

Очередь событий для монитора безопасности: все запросы от сущностей друг к другу должны отправляться только в неё

In [1]:
from multiprocessing import Queue
monitor_events_queue = Queue()

Зафиксируем формат сообщений

In [2]:
from dataclasses import dataclass


@dataclass
class Event:
    source: str       # отправитель
    destination: str  # получатель
    operation: str    # чего хочет (запрашиваемое действие)
    parameters: str   # с какими параметрами

### Монитор безопасности

Ниже в методе _check_policies можно увидеть пример политики безопасности:

```python
if event.source == "ControlSystem" \
        and event.destination == "LightsGPIO" \
        and event.operation == "set_mode" \
        and self._check_mode(event.operation):
    authorized = True
```            

в этом примере проверяется отправитель сообщения, получатель, запрашиваемая операция и даже параметры операции. Это максимально жёсткий вариант, очевидно, в зависимости от ситуации количество проверок можно уменьшить.

А пока это место для экспериментов, как можно из монитора безопасности заблокировать взаимодействие между сущностями.

In [3]:
from multiprocessing import Queue, Process
from multiprocessing.queues import Empty
import json

# формат управляющих команд для монитора
@dataclass
class ControlEvent:
    operation: str

# список разрешенных сочетаний сигналов светофора
# любые сочетания, отсутствующие в этом списке, запрещены
traffic_lights_allowed_configurations = [
    {"direction_1": "red", "direction_2": "green", "direction_1_arrow" : "off", "direction_2_arrow" : "on"},
    {"direction_1": "green", "direction_2": "red", "direction_1_arrow" : "on", "direction_2_arrow" : "off"},
    {"direction_1": "red", "direction_2": "red", "direction_1_arrow" : "off", "direction_2_arrow" : "off"},
    {"direction_1": "red", "direction_2": "yellow", "direction_1_arrow" : "off", "direction_2_arrow" : "on"},
    {"direction_1": "yellow", "direction_2": "yellow", "direction_1_arrow" : "on", "direction_2_arrow" : "on"},
    {"direction_1": "off", "direction_2": "off", "direction_1_arrow" : "off", "direction_2_arrow" : "off"},
    {"direction_1": "green", "direction_2": "red", "direction_1_arrow" : "on", "direction_2_arrow" : "off"},
    {"direction_1": "green", "direction_2": "yellow", "direction_1_arrow" : "on", "direction_2_arrow" : "on"},
    {"direction_1": "yellow_blinking", "direction_2": "yellow_blinking", "direction_1_arrow" : "off", "direction_2_arrow" : "off"},

]


# Класс, реализующий поведение монитора безопасности
class Monitor(Process):

    def __init__(self, events_q: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        self._events_q = events_q  # очередь событий для монитора (входящие сообщения)
        self._control_q = Queue()  # очередь управляющих команд (например, для остановки монитора)
        self._entity_queues = {}   # словарь очередей известных монитору сущностей
        self._force_quit = False   # флаг завершения работы монитора

    # регистрация очереди новой сущности
    def add_entity_queue(self, entity_id: str, queue: Queue):
        print(f"[монитор] регистрируем сущность {entity_id}")
        self._entity_queues[entity_id] = queue

    def _check_mode(self, mode_str: str) -> bool:
        mode_ok = False
        try:
            # извлечём структуру из строки, в случае ошибки запретим изменение режима
            mode = json.loads(mode_str)
            # проверим входит ли запрашиваемый режим в список разрешённых
            print(f"[монитор] проверяем конфигурацию {mode}")
            if mode in traffic_lights_allowed_configurations:
                # такой режим найден, можно активировать
                mode_ok = True
        except:
            mode_ok = False
        return mode_ok



    # проверка политик безопасности
    def _check_policies(self, event):
        print(f'[монитор] обрабатываем событие {event}')

        # default deny: всё, что не разрешено, запрещено по умолчанию!
        authorized = False

        # проверка на входе, что это экземпляр класса Event,
        # т.е. имеет ожидаемый формат
        if not isinstance(event, Event):
            return False

        #
        #  политики безопасности
        #

        # пример политики безопасности
        if event.source == "ControlSystem" \
                and event.destination == "LightsGPIO" \
                and event.operation == "set_mode" \
                and self._check_mode(event.parameters):
            authorized = True

        if event.source == "SelfDiagnostics" \
              and event.destination == "LightsGPIO" \
              and event.operation == "send_conditions" :
            authorized = True

        if event.source == "CitySystemConnector" \
                and event.destination == "ControlSystem" \
                and event.operation == "set_yellow_blinking" :
            authorized = True

        if event.source == "LightsGPIO" \
                and event.destination == "CitySystemConnector" \
                and event.operation == "send_conditions" :
            authorized = True



        if authorized is False:
            print("[монитор] событие не разрешено политиками безопасности")
        return authorized

    # выполнение разрешённого запроса
    # метод должен вызываться только после проверки политик безопасности
    def _proceed(self, event):
        print(f'[монитор] отправляем запрос {event}')
        try:
            # найдём очередь получателя события
            dst_q: Queue = self._entity_queues[event.destination]
            # и положим запрос в эту очередь
            dst_q.put(event)
        except  Exception as e:
            # например, запрос пришёл от или для неизвестной сущности
            print(f"[монитор] ошибка выполнения запроса {e}")

    # основной код работы монитора безопасности
    def run(self):
        print('[монитор] старт')

        # в цикле проверяет наличие новых событий,
        # выход из цикла по флагу _force_quit
        while self._force_quit is False:
            event = None
            try:
                # ожидание сделано неблокирующим,
                # чтобы можно было завершить работу монитора,
                # не дожидаясь нового сообщения
                event = self._events_q.get_nowait()
                # сюда попадаем только в случае получение события,
                # теперь нужно проверить политики безопасности
                authorized = self._check_policies(event)
                if authorized:
                    # если политиками запрос авторизован - выполняем
                    self._proceed(event)
            except Empty:
                # сюда попадаем, если новых сообщений ещё нет,
                # в таком случае немного подождём
                sleep(0.5)
            except Exception as e:
                # что-то пошло не так, выведем сообщение об ошибке
                print(f"[монитор] ошибка обработки {e}, {event}")
            self._check_control_q()
        print('[монитор] завершение работы')

    # запрос на остановку работы монитора безопасности для завершения работы
    # может вызываться вне процесса монитора
    def stop(self):
        # поскольку монитор работает в отдельном процессе,
        # запрос помещается в очередь, которая проверяется из процесса монитора
        request = ControlEvent(operation='stop')
        self._control_q.put(request)

    # проверка наличия новых управляющих команд
    def _check_control_q(self):
        try:
            request: ControlEvent = self._control_q.get_nowait()
            print(f"[монитор] проверяем запрос {request}")
            if isinstance(request, ControlEvent) and request.operation == 'stop':
                # поступил запрос на остановку монитора, поднимаем "красный флаг"
                self._force_quit = True
        except Empty:
            # никаких команд не поступило, ну и ладно
            pass

### Сущность ControlSystem

Эта сущность отправляет сообщение для другой сущности (LightsGPIO)

In [4]:
from multiprocessing import Queue, Process
import json


class ControlSystem(Process):

    def __init__(self, monitor_queue: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        # мы знаем только очередь монитора безопасности для взаимодействия с другими сущностями
        # прямая отправка сообщений в другую сущность запрещена в концепции FLASK
        self.monitor_queue = monitor_queue
        # создаём собственную очередь, в которую монитор сможет положить сообщения для этой сущности
        self._own_queue = Queue()
        self.duration = 3
        self.number_of_works = 6

        self.traffic_lights_queue = [
            {"direction_1": "red", "direction_2": "green", "direction_1_arrow" : "off", "direction_2_arrow" : "on"},
            {"direction_1": "green", "direction_2": "red", "direction_1_arrow" : "on", "direction_2_arrow" : "off"},
        ]


    # выдаёт собственную очередь для взаимодействия
    def entity_queue(self):
        return self._own_queue

    # основной код сущности

    def run(self):
        print(f'[{self.__class__.__name__}] старт')
        attempts = 150
        condition = 0
        while attempts > 0:
            try:
                event: Event = self._own_queue.get_nowait()

                if event.operation == "set_yellow_blinking":
                    print(f"\n[{self.__class__.__name__}] {event.source} ПРИКАЗ {event.parameters}")
                    print(f"[{self.__class__.__name__}] новый режим: {event.parameters}!\n")
                    self.conditions = event.parameters
                    sleep(30)
                    attempts -= 1
            except Empty:

                mode = self.traffic_lights_queue[condition]

                event = Event(source=self.__class__.__name__,
                          destination='LightsGPIO',
                          operation='set_mode',
                          parameters=json.dumps(mode)
                          )
                self.monitor_queue.put(event)
                if condition == 0:
                    condition = 1
                else:
                    condition = 0

                sleep(5)
                attempts -= 1
        print(f'[{self.__class__.__name__}] завершение работы')

### Сущность LightsGPIO

Эта сущность ждёт входящее сообщение в течение заданного периода времени, если получает - обрабатывает и завершает работу или выходит по таймауту.

In [5]:
from multiprocessing import Queue, Process
from time import sleep


class LightsGPIO(Process):

    def __init__(self, monitor_queue: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        # мы знаем только очередь монитора безопасности для взаимодействия с другими сущностями
        # прямая отправка сообщений в другую сущность запрещена в концепции FLASK
        self.monitor_queue = monitor_queue
        # создаём собственную очередь, в которую монитор сможет положить сообщения для этой сущности
        self._own_queue = Queue()

        self.conditions = {"direction_1": "off", "direction_2": "off", "direction_1_arrow" : "off", "direction_2_arrow" : "off"}

    def entity_queue(self):
        return self._own_queue

    # основной код сущности
    def run(self):
        print(f'[{self.__class__.__name__}] старт')
        attempts = 100
        while attempts > 0:
            try:
                event: Event = self._own_queue.get_nowait()
                if event.operation == "set_mode":
                    print(f"\n[{self.__class__.__name__}] {event.source} запрашивает изменение режима {event.parameters}")
                    print(f"[{self.__class__.__name__}] новый режим: {event.parameters}!\n")
                    self.conditions = event.parameters

                    mode_6 = self.conditions
                    event_6 = Event(source=self.__class__.__name__,
                          destination='CitySystemConnector',
                          operation='send_conditions',
                          parameters=json.dumps(mode_6)
                          )
                    self.monitor_queue.put(event_6)


                    sleep(0.2)
                    attempts -= 1
                if event.operation == "send_conditions":
                    print(f"\n[{self.__class__.__name__}] {event.source} запрашивает состояние режима {self.conditions}")
                    print(f"[{self.__class__.__name__}] отправленные сведения: {self.conditions}!\n")
                    sleep(0.2)
                    attempts -= 1
            except Empty:
                sleep(0.2)
                attempts -= 1
        print(f'[{self.__class__.__name__}] завершение работы\n')

###Сущность SelfDiagnostics
сущность само-диагностики

In [6]:
from multiprocessing import Queue, Process
import json


class SelfDiagnostics(Process):

    def __init__(self, monitor_queue: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        # мы знаем только очередь монитора безопасности для взаимодействия с другими сущностями
        # прямая отправка сообщений в другую сущность запрещена в концепции FLASK
        self.monitor_queue = monitor_queue
        # создаём собственную очередь, в которую монитор сможет положить сообщения для этой сущности
        self._own_queue = Queue()
        self.time_to_check = 8

    def entity_queue(self):
        return self._own_queue

    # основной код сущности
    def run(self):
      print(f'[{self.__class__.__name__}] старт')
      sleep(self.time_to_check)

      mode_4 = {}
      event_4 = Event(source=self.__class__.__name__,
            destination='LightsGPIO',
            operation='send_conditions',
            parameters=json.dumps(mode_4)
            )


      self.monitor_queue.put(event_4)
      sleep(30)


      print(f'[{self.__class__.__name__}] завершение работы')

###Сущность CitySystemConnector
сущность CitySystemConnector

In [7]:
from multiprocessing import Queue, Process
import json


class CitySystemConnector(Process):

    def __init__(self, monitor_queue: Queue):
        # вызываем конструктор базового класса
        super().__init__()
        # мы знаем только очередь монитора безопасности для взаимодействия с другими сущностями
        # прямая отправка сообщений в другую сущность запрещена в концепции FLASK
        self.monitor_queue = monitor_queue
        # создаём собственную очередь, в которую монитор сможет положить сообщения для этой сущности
        self._own_queue = Queue()
        self.time_to_send = 15

    def entity_queue(self):
        return self._own_queue

    # основной код сущности
    def run(self):
      print(f'[{self.__class__.__name__}] старт')
      attempts = 100
      while attempts > 0:
          try:
              event: Event = self._own_queue.get_nowait()
              if event.operation == "send_conditions":
                  print(f"\n[{self.__class__.__name__}] {event.source} имеет состояние {self.conditions}\n")
                  sleep(0.2)
                  attempts -= 1
          except Empty:
              pass
              if attempts == 50:
                  mode_4 = {"direction_1": "yellow_blinking", "direction_2": "yellow_blinking", "direction_1_arrow" : "off", "direction_2_arrow" : "off"}
                  event_4 = Event(source=self.__class__.__name__,
                        destination='ControlSystem',
                        operation='set_yellow_blinking',
                        parameters=json.dumps(mode_4)
                        )

                  self.monitor_queue.put(event_4)
              sleep(0.2)
              attempts -= 1

      sleep(30)
      print(f'[{self.__class__.__name__}] завершение работы')


### Инициализируем монитор и сущности

In [8]:
monitor = Monitor(monitor_events_queue)
control_system = ControlSystem(monitor_events_queue)
lights_gpio = LightsGPIO(monitor_events_queue)
self_diagnostics = SelfDiagnostics(monitor_events_queue)
city_system_connector = CitySystemConnector(monitor_events_queue)

регистрируем очереди сущностей в мониторе

In [9]:
monitor.add_entity_queue(control_system.__class__.__name__, control_system.entity_queue())
monitor.add_entity_queue(lights_gpio.__class__.__name__, lights_gpio.entity_queue())
monitor.add_entity_queue(self_diagnostics.__class__.__name__,self_diagnostics.entity_queue())
monitor.add_entity_queue(city_system_connector.__class__.__name__,city_system_connector.entity_queue())

[монитор] регистрируем сущность ControlSystem
[монитор] регистрируем сущность LightsGPIO
[монитор] регистрируем сущность SelfDiagnostics
[монитор] регистрируем сущность CitySystemConnector


### Запускаем всё

Ожидаемая последовательность событий

![Диаграмма последовательности вызовов](https://www.plantuml.com/plantuml/png/dPBVIiCm6CNlynIvdtk1NSZ02n4KXJr1w886-cUqcR0xgoBpIdoJLgqhtRg-mfStygJPM3js9OLyoPTpVia97ITQn7eU-6o6gZmr4w7c5r6euyYVB18j0ouIxhb6JpIHtZnMUd4JXKf7iPK5RjgJNQlx1vrStbtTMeNVhXZR0VdmV6yQ34QSQjhI5sNcWrE5QMrUgQHlysAUq7oZqcxyq1hbW6KxG8S5KWEBPHMe5MLeOBa6uHd4YWbVSs1JQg1OKktEF3ATSTI2Vk7Os6b6AupGerctn3uM5WWfn_OAxGRGr4OogTtktjCzmt3utyZ2q-fHQBb_JrSEv177oHigRB1E2ChOL1vxfPz8ZWjdBhv9ELn5Bw_vfFf4L2fFlZsEtkBBpNjxWH8mkyHWbbZbC1TCXbCsne1Vxmy0)

In [10]:
monitor.start()
control_system.start()
lights_gpio.start()
self_diagnostics.start()
city_system_connector.start()
sleep(30)

[ControlSystem] старт[монитор] старт

[LightsGPIO] старт
[SelfDiagnostics] старт[CitySystemConnector] старт

[монитор] обрабатываем событие Event(source='ControlSystem', destination='LightsGPIO', operation='set_mode', parameters='{"direction_1": "red", "direction_2": "green", "direction_1_arrow": "off", "direction_2_arrow": "on"}')
[монитор] проверяем конфигурацию {'direction_1': 'red', 'direction_2': 'green', 'direction_1_arrow': 'off', 'direction_2_arrow': 'on'}
[монитор] отправляем запрос Event(source='ControlSystem', destination='LightsGPIO', operation='set_mode', parameters='{"direction_1": "red", "direction_2": "green", "direction_1_arrow": "off", "direction_2_arrow": "on"}')

[LightsGPIO] ControlSystem запрашивает изменение режима {"direction_1": "red", "direction_2": "green", "direction_1_arrow": "off", "direction_2_arrow": "on"}
[LightsGPIO] новый режим: {"direction_1": "red", "direction_2": "green", "direction_1_arrow": "off", "direction_2_arrow": "on"}!

[монитор] обрабатыва

Process CitySystemConnector-5:
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "<ipython-input-7-ec8412e65fd2>", line 28, in run
    print(f"\n[{self.__class__.__name__}] {event.source} имеет состояние {self.conditions}\n")
                                                                          ^^^^^^^^^^^^^^^
AttributeError: 'CitySystemConnector' object has no attribute 'conditions'


[монитор] обрабатываем событие Event(source='ControlSystem', destination='LightsGPIO', operation='set_mode', parameters='{"direction_1": "green", "direction_2": "red", "direction_1_arrow": "on", "direction_2_arrow": "off"}')
[монитор] проверяем конфигурацию {'direction_1': 'green', 'direction_2': 'red', 'direction_1_arrow': 'on', 'direction_2_arrow': 'off'}
[монитор] отправляем запрос Event(source='ControlSystem', destination='LightsGPIO', operation='set_mode', parameters='{"direction_1": "green", "direction_2": "red", "direction_1_arrow": "on", "direction_2_arrow": "off"}')

[LightsGPIO] ControlSystem запрашивает изменение режима {"direction_1": "green", "direction_2": "red", "direction_1_arrow": "on", "direction_2_arrow": "off"}
[LightsGPIO] новый режим: {"direction_1": "green", "direction_2": "red", "direction_1_arrow": "on", "direction_2_arrow": "off"}!

[монитор] обрабатываем событие Event(source='LightsGPIO', destination='CitySystemConnector', operation='send_conditions', paramet

### Теперь останавливаем

In [11]:
monitor.stop()
control_system.join()
lights_gpio.join()
self_diagnostics.join()
city_system_connector.join()
monitor.join()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-73a58d5c3abb>", line 2, in <cell line: 0>
    control_system.join()
  File "/usr/lib/python3.11/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/popen_fork.py", line 43, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/popen_fork.py", line 27, in poll
    pid, sts = os.waitpid(self.pid, flag)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line

TypeError: object of type 'NoneType' has no len()

## Заключение

В этом блокноте продемонстрирован базовый функционал контролируемого изменения режима работы светофора.

В примере не реализованы некоторые сущности и большая часть логики работы светофора, которую можно предположить по архитектурной диаграмме. Попробуйте сделать это самостоятельно!

## Упражнения

Уровень "Новичок"

- в коде ControlSystem измените режим на недопустимый (два зелёных) и выполните все ячейки. Убедитесь, что монитор безопасности заблокировал сообщение, как нарушающее политику безопасности
- измените политики безопасности так, чтобы был возможен режим "моргающий жёлтый" (yellow_blinking), переводящий перекрёсток в режим нерегулируемого

Уровень "Средней сложности"

- добавьте политики безопасности для доп. секций со стрелками (поворот налево или направо)
- измените код сущностей, чтобы они не завершали работу после одного сообщения, а работали произвольное время (см. реализацию монитора безопасности)
- реализуйте сущность само-диагностики (SelfDiagnostics) и отправку сообщений от LightsGPIO (необходимо доработать политики безопасности!)

Уровень "Продвинутый"

- измените код сущности ControlSystem, реализуйте смену режимов по таймеру (заданную длительность зелёного по каждому направлению)
- реализуйте сущность CitySystemConnector, которая будет имитировать получение изменения режима, реализуйте взаимодействие CitySystemConnector и ControlSystem (понадобится доработать политики безопасности). Например, изменение длительности зелёного по направлениям или отключение светофора (перевод перекрёстка в режим нерегулируемого)
- реализуйте передачу в компонент CitySystemConnector информации об исправности светофора (статус самодиагностики; текущий режим работы). В компоненте реализуйте вывод в виде сообщений о состоянии системы